In [16]:
import pandas as pd

paged_data = pd.read_csv('q5_pilot_result_2.csv')
group_keys = paged_data["n_name"].unique()
print("n_name groups:", group_keys)
paged_data

n_name groups: ['CHINA                    ' 'INDIA                    '
 'INDONESIA                ' 'JAPAN                    '
 'VIETNAM                  ']


,n_name,revenue,o_pageid,l_pageid
0,CHINA,56313.2160,7557.0,32514.0
1,CHINA,28878.9300,215949.0,929135.0
2,CHINA,58016.5222,219261.0,943397.0
3,CHINA,44944.6437,242569.0,1043672.0
4,CHINA,10745.5260,303902.0,1307582.0
...,...,...,...,...
3434,VIETNAM,83333.6322,26625233.0,114579438.0
3435,VIETNAM,31855.4640,26655567.0,114709922.0
3436,VIETNAM,67462.5420,26740427.0,115075271.0
3437,VIETNAM,22054.5000,26804844.0,115352475.0


In [17]:
n_pages_l = paged_data.groupby("n_name").agg({"l_pageid": "nunique"})["l_pageid"]
n_pages_o = paged_data.groupby("n_name").agg({"o_pageid": "nunique"})["o_pageid"]
print("n_pages_l:\n", n_pages_l)
print("n_pages_o:\n", n_pages_o)

n_pages_l:
 n_name
CHINA                        683
INDIA                        698
INDONESIA                    718
JAPAN                        668
VIETNAM                      669
Name: l_pageid, dtype: int64
n_pages_o:
 n_name
CHINA                        684
INDIA                        698
INDONESIA                    720
JAPAN                        668
VIETNAM                      669
Name: o_pageid, dtype: int64


In [24]:
import numpy as np
l_sum = paged_data.groupby(["n_name", "l_pageid"]).agg({"revenue": "sum"})["revenue"]
group_keys = paged_data["n_name"].unique()
group_l_vars = {}
for group_key in group_keys:
    group_l_sum = l_sum[group_key] / n_pages_o[group_key]
    group_l_sum = np.concatenate([group_l_sum, np.zeros(n_pages_l[group_key] - len(group_l_sum))])
    group_l_var = np.var(group_l_sum)
    print(f"{group_key} variance: {group_l_var}")
    group_l_vars[group_key] = group_l_var
max_l_var = max(group_l_vars.values())

CHINA                     variance: 1427.367356628544
INDIA                     variance: 1276.6858647596773
INDONESIA                 variance: 1289.8652865884526
JAPAN                     variance: 1562.2507018817566
VIETNAM                   variance: 1356.2781058334888


In [23]:
import numpy as np
o_sum = paged_data.groupby(["n_name", "o_pageid"]).agg({"revenue": "sum"})["revenue"]
group_keys = paged_data["n_name"].unique()
group_o_vars = {}
for group_key in group_keys:
    group_o_sum = o_sum[group_key] / n_pages_l[group_key]
    group_o_sum = np.concatenate([group_o_sum, np.zeros(n_pages_o[group_key] - len(group_o_sum))])
    group_o_var = np.var(group_l_sum)
    print(f"{group_key} variance: {group_o_var}")
    group_o_vars[group_key] = group_o_var
max_o_var = max(group_o_vars.values())

CHINA                     variance: 1356.2781058334888
INDIA                     variance: 1356.2781058334888
INDONESIA                 variance: 1356.2781058334888
JAPAN                     variance: 1356.2781058334888
VIETNAM                   variance: 1356.2781058334888


In [20]:
avg = paged_data.groupby("n_name").agg({"revenue": "sum"})
avgs = {}
for group_key in group_keys:
    avgs[group_key] = avg.loc[group_key]["revenue"] / n_pages_o[group_key] / n_pages_l[group_key]
    print(group_key, avgs[group_key])
min_avg = min(avgs.values())
print("min_avg", min_avg)

CHINA                     56.688041978329174
INDIA                     56.5339862043415
INDONESIA                 52.99061213865676
JAPAN                     58.98886386769874
VIETNAM                   56.5426830849873
min_avg 52.99061213865676


In [21]:
import math
math.sqrt(1562.2507018817566) / 52.99061213865676

0.7458928219853284

In [25]:
import scipy.stats
z = scipy.stats.norm.ppf(0.999)
n = z**2 * (max_l_var + max_o_var) / (min_avg**2) / (0.05**2)
print("n", n)

n 3970.163202463841


In [27]:
3970.163202463841/668 * 0.05

0.297167904376036